# Homework 2

In [1]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2018 term"

This homework covers material from the sentiment classification unit. The primary value of doing the work is that it provides more hands-on experience with the dataset and the models we explored. All the code you write has potential value in the bake-off for this unit as well.

Submission URL: https://canvas.stanford.edu/courses/83399/quizzes/50657

## Questions 1–4: Reproducing a Socher et al's NaiveBayes baseline [4 points]

[Socher et al. 2013](http://www.aclweb.org/anthology/D/D13/D13-1170.pdf) compare against a Naive Bayes baseline with bigram features. See how close you can come to reproducing the performance of this model on the binary, root-only problem (values in the rightmost column of their Table 1, rows 1 and 3).

Specific tasks:

1. Write a bigrams feature function, on the model of `unigrams_phi`. Call this `bigrams_phi`. In writing this function, ensure that each example is padded with start and end symbols (say, `<S>` and `</S>`), so that these contexts are properly reflected in the feature space.

1. Write a function `fit_nb_classifier_with_crossvalidation` that serves as a wrapper for `sklearn.naive_bayes.MultinomialNB` and searches over these values for the smoothing parameter `alpha`: `[0.1, 0.5, 1.0, 2.0]`, using 3-fold cross-validation.

1. Use `sst.experiment` to run the experiments, assessing against `dev_reader`.

__To submit:__

1. Your `bigrams_phi`
1. Your `fit_nb_classifier`
1. Your call to `sst.experiment` 
1. The average F1 score that `sst.experiment` reported.

__A note on performance__: in our experience, the bigrams Naive Bayes model gets around 0.75. It's fine to submit answers with comparable numbers; the Socher et al. baselines are very strong. We're not evaluating how good your model is; we want to see your code, and we're interested to see what the range of F1 scores is across the whole class.

In [5]:
from collections import Counter

def emit_bigrams(unigrams):
    full_unigrams = ['<S>'] + unigrams + ['</S>']
    for i in range(len(full_unigrams)):
        if i+1 < len(full_unigrams):
            yield full_unigrams[i] + ' ' + full_unigrams[i+1]

def bigrams_phi(tree):
    return Counter(emit_bigrams(tree.leaves()))

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

def fit_nb_classifier_with_crossvalidation(X, y):
    model = MultinomialNB()
    parameters = {'alpha': [0.1, 0.5, 1.0, 2.0]}
    classifier = GridSearchCV(model, parameters, n_jobs=-1)
    
    classifier.fit(X, y)
    
    return classifier

In [8]:
from sst import experiment

experiment(
    bigrams_phi,
    fit_nb_classifier_with_crossvalidation
)

{'cv': None, 'error_score': 'raise', 'estimator__alpha': 1.0, 'estimator__class_prior': None, 'estimator__fit_prior': True, 'estimator': MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True), 'fit_params': None, 'iid': True, 'n_jobs': -1, 'param_grid': {'alpha': [0.1, 0.5, 1.0, 2.0]}, 'pre_dispatch': '2*n_jobs', 'refit': True, 'return_train_score': 'warn', 'scoring': None, 'verbose': 0}
Accuracy: 0.725
             precision    recall  f1-score   support

   negative      0.685     0.766     0.723       975
   positive      0.769     0.688     0.726      1101

avg / total      0.730     0.725     0.725      2076



0.7249441080550532

## Question 5–6: A more powerful vector-summing baseline [4 points]

In the section [Distributed representations as features](sst_03_neural_networks.ipynb#Distributed-representations-as-features), we looked at a baseline for the binary SST problem in which each example is modeled as the sum of its 50-dimensional GloVe representations. A `LogisticRegression` model was used for prediction. A neural network might do better here, since there might be complex relationships between the input feature dimensions that a linear classifier can't learn. 

To address this question, rerun the experiment with `tf_shallow_neural_classifier.TfShallowNeuralClassifier` as the classifier. Specs:
* Use `sst.experiment` to conduct the experiment. 
* Using 3-fold cross-validation, exhaustively explore this set of hyperparameter combinations:
  * The hidden dimensionality at 50, 100, and 200.
  * The hidden activation function as `tf.nn.tanh` or `tf.nn.relu`.
* (For all other parameters to `TfShallowNeuralClassifier`, use the defaults.)

__To submit:__

* Your average F1 score according to `sst.experiment`. 
* The optimal hyperparameters chosen in your experiment. (You can just paste in the dict that `sst._experiment` prints.)

No need to include your supporting code. 

We're not evaluating the quality of your model. (We've specified the protocols completely, but there will still be a  lot of variation in the results.) However, the primary goal of this question is to get you thinking more about this strikingly good baseline feature representation scheme for SST, so we're sort of hoping you feel compelled to try out variations on your own.

In [11]:
import utils
import os

glove_home = 'vsmdata/glove.6B'

glove_lookup = utils.glove2dict(
    os.path.join(glove_home, 'glove.6B.50d.txt'))

In [13]:
import numpy as np

def vsm_leaves_phi(tree, lookup, np_func=np.sum):
    """Represent `tree` as a combination of the vector of its words.
    
    Parameters
    ----------
    tree : nltk.Tree   
    lookup : dict
        From words to vectors.
    np_func : function (default: np.sum)
        A numpy matrix operation that can be applied columnwise, 
        like `np.mean`, `np.sum`, or `np.prod`. The requirement is that 
        the function take `axis=0` as one of its arguments (to ensure
        columnwise combination) and that it return a vector of a 
        fixed length, no matter what the size of the tree is.
    
    Returns
    -------
    np.array, dimension `X.shape[1]`
            
    """      
    allvecs = np.array([lookup[w] for w in tree.leaves() if w in lookup])    
    if len(allvecs) == 0:
        dim = len(next(iter(lookup.values())))
        feats = np.zeros(dim)
    else:       
        feats = np_func(allvecs, axis=0)      
    return feats

def glove_leaves_phi(tree, np_func=np.sum):
    return vsm_leaves_phi(tree, glove_lookup, np_func=np_func)

In [28]:
from tf_shallow_neural_classifier import TfShallowNeuralClassifier
import tensorflow as tf

from sklearn.metrics import f1_score, make_scorer

def fit_shallow_nn(X, y):
    model = TfShallowNeuralClassifier(max_iter=50)
    parameters = {'hidden_dim': [50, 100, 200], 'hidden_activation': [tf.nn.tanh, tf.nn.relu]}
    scorer = make_scorer(f1_score, pos_label='positive')
    classifier = GridSearchCV(model, parameters, scoring=scorer, n_jobs=1)
    
    classifier.fit(X, y)
    
    means = classifier.cv_results_['mean_test_score']
    stds = classifier.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, classifier.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    
    return classifier

In [29]:
experiment(glove_leaves_phi, fit_shallow_nn, vectorize=False)

/Users/mwilber/.pyenv/versions/3.6.1/lib/python3.6/site-packages/tensorflow/python/client/session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
Iteration 50: loss: 2.8569328188896184

0.652 (+/-0.040) for {'hidden_activation': <function tanh at 0x11fa65bf8>, 'hidden_dim': 50}
0.693 (+/-0.077) for {'hidden_activation': <function tanh at 0x11fa65bf8>, 'hidden_dim': 100}
0.690 (+/-0.137) for {'hidden_activation': <function tanh at 0x11fa65bf8>, 'hidden_dim': 200}
0.672 (+/-0.066) for {'hidden_activation': <function relu at 0x11f988e18>, 'hidden_dim': 50}
0.696 (+/-0.040) for {'hidden_activation': <function relu at 0x11f988e18>, 'hidden_dim': 100}
0.710 (+/-0.022) for {'hidden_activation': <function relu at 0x11f988e18>, 'hidden_dim': 200}
Accuracy: 0.695
             precision    recall  f1-score   support

   negative      0.638     0.851     0.729      1002
   positive      0.798     0.549     0.651      1074

avg / total      0.721     0.695     0.689      2076



0.690113231431255

## Questions 7–8: Bidirectional RNN [2 points]

The auxiliary notebook `tensorflow_models.ipynb` [subclasses TfRNNClassifier with a bidirectional RNN](tensorflow_models.ipynb#A-bidirectional-RNN-Classifier). In this model, the RNN is run in both directions and the concatenation of the two final states is used as the basis for the classification decision. Evaluate this model against the SST dev set. You can set up the model however you wish for this.

__To submit:__

* Your call to `TfBidirectionalRNNClassifier` (so that we can see the hyperparmeters you chose).
* Your average F1 score according to a `classification_report` on the `dev` set.

As above, we will not evaluate you based on how good your F1 score is. You just need to submit one. __There is even value in seeing what really doesn't work__, so low scores have interest!

In [31]:
import sst

X_rnn_train, y_rnn_train = sst.build_binary_rnn_dataset(sst.train_reader)
sst_train_vocab = sst.get_vocab(X_rnn_train, n_words=3000)

In [36]:
from tf_rnn_classifier import TfRNNClassifier

class TfBidirectionalRNNClassifier(TfRNNClassifier):
    
    def build_graph(self):
        self._define_embedding()

        self.inputs = tf.placeholder(
            tf.int32, [None, self.max_length])

        self.ex_lengths = tf.placeholder(tf.int32, [None])

        # Outputs as usual:
        self.outputs = tf.placeholder(
            tf.float32, shape=[None, self.output_dim])

        # This converts the inputs to a list of lists of dense vector
        # representations:
        self.feats = tf.nn.embedding_lookup(
            self.embedding, self.inputs)

        # Same cell structure as the base class, but we have
        # forward and backward versions:
        self.cell_fw = tf.nn.rnn_cell.LSTMCell(
            self.hidden_dim, activation=self.hidden_activation)
        
        self.cell_bw = tf.nn.rnn_cell.LSTMCell(
            self.hidden_dim, activation=self.hidden_activation)

        # Run the RNN:
        outputs, finals = tf.nn.bidirectional_dynamic_rnn(
            self.cell_fw,
            self.cell_bw,
            self.feats,
            dtype=tf.float32,
            sequence_length=self.ex_lengths)
      
        # finals is a pair of `LSTMStateTuple` objects, which are themselves
        # pairs of Tensors (x, y), where y is the output state, according to
        # https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMStateTuple
        # Thus, we want the second member of these pairs:
        last_fw, last_bw = finals          
        last_fw, last_bw = last_fw[1], last_bw[1]
        
        last = tf.concat((last_fw, last_bw), axis=1)
        
        self.feat_dim = self.hidden_dim * 2               

        # Softmax classifier on the final hidden state:
        self.W_hy = self.weight_init(
            self.feat_dim, self.output_dim, 'W_hy')
        self.b_y = self.bias_init(self.output_dim, 'b_y')
        self.model = tf.matmul(last, self.W_hy) + self.b_y    

In [37]:
def fit_bidirectional_rnn(X, y):
    model = TfBidirectionalRNNClassifier(sst_train_vocab)
    
    return model.fit(X, y)

experiment(glove_leaves_phi, fit_bidirectional_rnn, vectorize=False)

Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead


Iteration 100: loss: 3.460171163082123

Accuracy: 0.513
             precision    recall  f1-score   support

   negative      0.000     0.000     0.000      1010
   positive      0.513     1.000     0.679      1066

avg / total      0.264     0.513     0.348      2076



0.33927434754933167